In [ ]:
!pip install tweepy

In [ ]:
import os
import tweepy as tw
import pandas as pd
from tqdm import tqdm, notebook

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
consumer_api_key = user_secrets.get_secret("twitter_api_key")
consumer_api_secret = user_secrets.get_secret("twitter_api_key_secret Secret")

In [ ]:
howmany = 10000

In [ ]:
keywords = ["#ChatGPT", "#GPT3", "#GPT", "#LLMs", "#Llama", "#LLM" "#GPT4",  "#GPTs", "#StableLM",
           "#Bard", "#GPTNeo", "#autogpt", "#langchain", "#vicuna", "#Bing"]

In [ ]:
auth = tw.OAuthHandler(consumer_api_key, consumer_api_secret)

api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

# minus a month
currentTimeDate = datetime.now() - relativedelta(days=1)
currentTime = currentTimeDate.strftime('%Y-%m-%d')
print(currentTime)

In [ ]:
tweets_df = pd.DataFrame()

for kw in keywords: 
    search_words = kw + " -filter:retweets"
    date_since = str(currentTime)

    # Collect tweets
    tweets = tw.Cursor(api.search_tweets,
                  q=search_words,
                  lang="en",
                since=date_since).items(howmany)

    tweets_copy = []
    for tweet in tqdm(tweets):
         tweets_copy.append(tweet)

    tweets_df1 = pd.DataFrame()
    for tweet in tqdm(tweets_copy):
        hashtags = []
        try:
            for hashtag in tweet.entities["hashtags"]:
                hashtags.append(hashtag["text"])
            text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
        except:
            pass
        tweets_df1 = tweets_df1.append(pd.DataFrame({'user_name': tweet.user.name, 
                                                'text': text, 
                                               'user_location': tweet.user.location,
                                               'user_description': tweet.user.description,
                                               'user_created': tweet.user.created_at,
                                               'user_followers': tweet.user.followers_count,
                                               'user_friends': tweet.user.friends_count,
                                               'user_favourites': tweet.user.favourites_count,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                                'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source,
                                              # 'is_retweet': tweet.retweeted
                                             }
                                                  , index=[0]))
    tweets_df = pd.concat([tweets_df, tweets_df1], axis = 0)
    print(tweets_df.shape)

In [ ]:
tweets_df.to_csv('data/tweets.csv', index = False)